## 0 - Setup

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy.optimize import minimize

## 1 - Importing Data

In [2]:
mens_df = pd.read_csv('../data/mens.csv',header=0,parse_dates=["Date"])
womens_df = pd.read_csv('../data/womens.csv',header=0,parse_dates=["Date"])

# Remove walkovers
mens_df = mens_df[mens_df['Comment']!='Walkover']
womens_df = womens_df[womens_df['Comment']!='Walkover']

K:\Programs\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (11,12,15,16,17,18,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
K:\Programs\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (25,28,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 2 - Theoretical Model Derivation

Consider a universe of n players (e.g all players in an upcoming tournament) and introduce the notion of the ability parameter for the ith player $ \alpha_i(S,t) $ where <br>
* $\alpha_i$ - is some real valued notion of the ith player's ability
* S - is the surface the game is played on
* t - is time
<br>

Then by the notion of the simple Bradley Terry model we suppose that
$$ P(player\,i\,beats\,player\,j) \propto  \frac{\alpha_i(S,t)}{\alpha_i(S,t)+\alpha_j(S,t)}$$

Now consider a tennis match between players i & j. Suppose the match ends with player i winning $g_i$ games and player j winning $g_j$ games. Then supposing each game is independant the probability that this happens is

$$P(match\,ends\,g_i:g_j) = \frac{\alpha_i(S,t)^{g_i}\alpha_j(S,t)^{g_j}}{(\alpha_i(S,t)+\alpha_j(S,t))^{g_i+g_j}} $$ 

This allows us to define the following likelihood function for $\underline{\alpha(S,t)}=(\alpha_1(S,t),...,\alpha_n(S,t))$<br><br>
$$L(\underline{\alpha(S,t)}) = \prod_{k\in A_t}{\frac{\alpha_i(S,t)^{g_i}\alpha_j(S,t)^{g_j}}{(\alpha_i(S,t)+\alpha_j(S,t))^{g_i+g_j}}}$$

Where we abuse notation slightly, to say that the indecies i & j depend on k. So the product is over all games in the data set, where the indecies i & j simply mean the players in the kth game

Now we make the following simplification $ \alpha_i(S,t) = \alpha_i(S) $, that is we assume the ability parameters are time independant. This is justified as follows

* we expect pro players to be on the upper end of the [skill learning curve](https://www.valamis.com/documents/10197/520324/s-curve-lc.png)
* we expect that underlying player performance won't change rapidly in time, say as a beginner's ability would (excluding injury)

We do expect that a player's ability varies over the course of their career, so to account for this we attach a time weight to the data. i.e more recent games are more important. This allows us to update the likelihood function to<br>

$$L(\underline{\alpha(S,t)}) = \prod_{k\in A_t}{\left(\frac{\alpha_i^{g_i}\alpha_j^{g_j}}{(\alpha_i+\alpha_j)^{g_i+g_j}}\right)^{exp(\epsilon(t-t_k))}}$$

<br>
Where we drop the $(S,t)$ from all the $\alpha's$

Now reparametrising the $\alpha's$ as
$$  \alpha_i(S,t) = exp\left(s_i w_i(S)\right)$$ 
Where $s_i$ is the player's skill parameter and $w_i(S)$ is a surface weighting function <br>

The log-likelihood function becomes 
$$\ell(\underline{\alpha(S,t)}) = \sum_{k\in A_t}{exp(\epsilon(t-t_k)) \left[     g_is_iw_i+g_js_jw_j - (g_i+g_j) \ln\left(e^{s_iw_i}+e^{s_jw_j}\right)    \right]}$$
To avoid over-parametrisation we need to add a constraint on the surface weight function

Each player has a suface weight function w_i(S) which weights a result based on a player's strength on that surface. We have three categories of surface; Hardcourt/carpet weighting/Greenset, Clay and Grass. <br>

We'll be training the model with the aim to predict the player's ability on some given surface S. Say we enumerate the surfaces as {0,1,2}. Then we only really have two weights to fit;

$$ s_i(S) =   \left\{
\begin{array}{ll}
      1 & S = \,data\,surface \\
      some\,weight & S \neq data\,surface \\
\end{array} 
\right.  $$

So we can say $s_i(S)\in\{1,\omega_1,\omega_2\} $ where $\omega_{1/2}$ are the weights for the player's performance on different surfaces. We constrain $0 \le \omega_{1/2} \leq 1.5 $ as pros tend to be somewhat consistent 

## 3 - Setup For Model

To prepare the data for model fitting we need to
* Calculate the epsilon param and the time interval we want to consider
* Prepare the df by removing unneeded cols and totalling up the number of games won
* Format the df by replacing player/surface names with indecies

1. Use half life time decay given in 2011 paper to calc how far back we look

In [3]:
def setup_params(time_decay_half_life = 240,significance_tol = 0.1,high_winning_prob = 0.7):
    eps = 1/(2*time_decay_half_life) # time decay epsilon
    max_time_interval = np.log(np.log(significance_tol)/np.log(high_winning_prob)) / eps # in days, which is how far back we'll consider
    
    return eps,max_time_interval

eps,max_time_interval = setup_params()
print("We'll consider ",max_time_interval/365," years worth of data")

We'll consider  2.4525539222882347  years worth of data


2. Total up the games won by each player and filter by time, remove unneeded cols

In [4]:
def prepare_frame(df,start_date,eps,max_time_interval,prediction_surface,filter_player_universe=False):
    # 1- Total up games won
    winner_cols = [c for c in df.columns if c[0]=="W" and any(char.isdigit() for char in c)]
    loser_cols =  [c for c in df.columns if c[0]=="L" and any(char.isdigit() for char in c)]
    df[winner_cols].fillna(0);df[loser_cols].fillna(0)
    df.loc[:,'gi']= df[winner_cols].sum(axis=1)
    df.loc[:,'gj']= df[loser_cols].sum(axis=1)
    # 2 - Add time weight col
    df['dt'] = (start_date - df['Date']).dt.days.astype('int16') # An integer amount of days
    df['time_decay'] = round(np.exp(-eps*df['dt']),2)
    # 3 - Filter players
    if filter_player_universe:
        tournament_df = df[df['Date']==start_date]
        players = set(np.concatenate([tournament_df['Winner'].values,tournament_df['Loser'].values],axis=0))
        df = df[df['Winner'].isin(players) | df['Loser'].isin(players)]
    # 4 - Fliter in time
    end_date = start_date - pd.Timedelta(days=max_time_interval)
    df = df[(df['Date']>=end_date)&(df['Date']<start_date)] # Strict ineq here important!
    # 5 - Filter cols
    df = df[['Surface','Winner','Loser','time_decay','gi','gj']]
    
    # 6 - Player dict
    unique_players = set(np.concatenate([df['Winner'].values,df['Loser'].values],axis=0))
    n = len(unique_players)
    player_dict = {}
    for i,player in enumerate(unique_players):
        player_dict[player] = i
    # 7 - surface dict
    if prediction_surface == 'Grass':
        surface_dict = defaultdict(lambda: 1, {'Clay':0,'Grass':-1})
    elif prediction_surface != 'Clay':
        surface_dict = defaultdict(lambda: -1, {'Clay':0,'Grass':1})
    else:
        surface_dict = defaultdict(lambda: 1, {'Clay':-1,'Grass':0})
    surface_dict # The prediction surface should have key -1
    # 8 - Mapping function
    def map_df_row_to_keys(row,player_dict,surface_dict):
        row['Winner'] = player_dict[row['Winner']]
        row['Loser'] = player_dict[row['Loser']]
        row['Surface'] = surface_dict[row['Surface']]
        row['Surface_mult'] = 0 if row['Surface'] == -1 else 1
        return row
    # 9 - Mapping and data typing
    formatted_df = df[[c for c in df.columns if 'Rank' not in c]].apply(lambda x : map_df_row_to_keys(x,player_dict,surface_dict),axis=1)
    formatted_df.loc[:,formatted_df.columns != 'time_decay'] = formatted_df.loc[:,formatted_df.columns != 'time_decay'].astype(int) # So we can use the entries as list indecies later on

    return formatted_df,player_dict,n

In [5]:
game_day = pd.to_datetime("2020-09-28")
mens_df.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Enqvist T.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Federer R.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Fromberg R.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
formatted_mens_df,player_dict,n = prepare_frame(mens_df,game_day,eps,max_time_interval,'Clay')
print(n,formatted_mens_df.shape)
formatted_mens_df.head()

374 (5203, 7)


,Surface,Winner,Loser,time_decay,gi,gj,Surface_mult
47826,-1,84,373,0.15,6,3,0
47827,-1,227,266,0.15,6,3,0
47828,-1,196,199,0.15,5,7,0
47829,-1,131,307,0.15,5,7,0
47830,-1,53,262,0.15,6,3,0


## 3 - Calculating Log-Liklihood

We now have the df in a nice format, we want to be able to compute the log lilkihood quickly. $$\ell(\underline{\alpha(S,t)}) = \sum_{k\in A_t}{exp(\epsilon(t-t_k)) \left[     g_is_iw_i+g_js_jw_j - (g_i+g_j) \ln\left(e^{s_iw_i}+e^{s_jw_j}\right)    \right]}$$

We want to do as much precomputation as possible. To use scipy.optimise we need to flatten our params. Let <br>
$x$ be a $(3n,1)$ list where <br>
* x[:n] - represents alpha <br>
* x[n:] - represents the surface weights
So that x[i] * x[n+i+S] is the weigted contribuion

In [7]:
# Initial values for the alphas and surface weights
x = np.ones(3*n)

In [8]:
def log_lilkihood(x,n,df):
    """
    x - a (3n,1) dimentional array with cols [alpha,W]
    n - an integer
    df - an (n,8) df
    """
    # Return -log_lilkihood, so we can max log_lilkihood by minimising -log_lilkihood
    df['aiwi'] = x[df['Winner']] * ((x[n+2*df['Winner']+df['Surface']]) ** df['Surface_mult'])
    df['ajwj'] = x[df['Loser']] *  ((x[n+2*df['Loser']+df['Surface']]) ** df['Surface_mult'])

    return -sum(df['time_decay']*(df['gi']*df['aiwi']+df['gj']*df['ajwj']-(df['gi']+df['gj'])*np.log(np.exp(df['aiwi'])+np.exp(df['ajwj']))))

In [9]:
%%time
log_lilkihood(np.ones((3*n)),n,formatted_mens_df)

Wall time: 8 ms


15660.191072729202

## 4 - Model Fitting

We now have the log_lilkihood function which takes in
* df - the formatted df processed above
* $\alpha$ - an nx1 vector
* W - an nx2 vector

We in total have 3n params to fit

In [10]:
print("We have ",formatted_mens_df.shape[0]," matches to fit ",3*n," params")

We have  5203  matches to fit  1122  params


In [11]:
x0=np.ones((3*n))
bds = [(0.1,2.5)]*n + [(0.1,1.5)]*(2*n)
options = {'disp':True,'maxiter':50}

In [ ]:
%%time
res = minimize(log_lilkihood,x0=x0,args=(n,formatted_mens_df),method='SLSQP',bounds=bds,options=options)

In [28]:
res.x

array([1., 1., 1., ..., 1., 1., 1.])

## 5 - Testing

In [14]:
alpha = res.x[:n]
player_rankings = {}
for key, value in player_dict.items():
    player_rankings[key]=alpha[value]

In [15]:
test_df = mens_df[mens_df['Date']==game_day]
test_df = test_df[test_df['Winner'].isin(player_dict.keys()) & test_df['Loser'].isin(player_dict.keys())]

In [16]:
def add_model_rankings(row,alpha):
    return alpha[player_dict[row['Winner']]],alpha[player_dict[row['Loser']]]

In [17]:
test_df[['W_alpha','L_alpha']]=test_df.apply(lambda x:add_model_rankings(x,alpha),axis=1,result_type="expand")
test_df = test_df[['Winner','Loser','W_alpha','L_alpha']]
test_df

,Winner,Loser,W_alpha,L_alpha
53065,Thiem D.,Cilic M.,1.464636,1.202261
53066,Nishioka Y.,Auger-Aliassime F.,0.980732,1.156257
53067,Sock J.,Opelka R.,0.922339,1.184489
53068,Gaston H.,Janvier M.,0.615583,0.856889
53070,Herbert P.H.,Mmoh M.,1.052487,1.152834
53071,Kukushkin M.,Fognini F.,0.906890,1.155853
53072,Sonego L.,Gomez E.,0.995701,0.957914
53073,Paul T.,Duckworth J.,0.925473,0.845359
53074,Mcdonald M.,Diez S.,0.923991,1.347876
53075,Giustino L.,Moutet C.,0.710592,1.192043


In [18]:
test_df['Correct'] = test_df['W_alpha'] > test_df['L_alpha']
test_df['d_alpha'] = np.abs(test_df['W_alpha'] - test_df['L_alpha'])
test_df['p'] = test_df[["W_alpha", "L_alpha"]].max(axis=1) / (test_df['W_alpha'] + test_df['L_alpha'])
test_df.sort_values(by='p',ascending=False)

,Winner,Loser,W_alpha,L_alpha,Correct,d_alpha,p
53082,Nadal R.,Gerasimov E.,1.953501,0.924490,True,1.029011,0.678772
53075,Giustino L.,Moutet C.,0.710592,1.192043,False,0.481451,0.626522
53079,Ruud C.,Sugita Y.,1.427481,0.924956,True,0.502525,0.606810
53088,Altmaier D.,Lopez F.,0.648764,0.971107,False,0.322343,0.599496
53084,Sandgren T.,Hurkacz H.,0.902359,1.350537,False,0.448178,0.599467
53074,Mcdonald M.,Diez S.,0.923991,1.347876,False,0.423884,0.593290
53081,Bublik A.,Monfils G.,1.304967,0.895689,True,0.409278,0.592990
53077,Struff J.L.,Tiafoe F.,1.142774,0.809428,True,0.333346,0.585377
53068,Gaston H.,Janvier M.,0.615583,0.856889,False,0.241305,0.581939
53067,Sock J.,Opelka R.,0.922339,1.184489,False,0.262151,0.562215


In [19]:
# Accuracy on all games
print(test_df['Correct'].sum()/test_df.shape[0],test_df.shape[0])

# Accuracy on confident games
confident_test_df = test_df[test_df['p']>0.6]
print(confident_test_df['Correct'].sum()/confident_test_df.shape[0],confident_test_df.shape[0])

0.43478260869565216 23
0.6666666666666666 3
